In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv
import pandas as pd 
application_df = pd.read_csv('../Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [3]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = application_values[application_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [4]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_class = classification_values[classification_values < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other')

In [5]:
# ASK_AMT bins
ask_values_0 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000]
ask_values_1 = application_df['ASK_AMT'][(application_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '>5000')

In [6]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [7]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other')

In [8]:
# Generate the categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\peter\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

C:\Users\peter\AppData\Local\Temp\ipykernel_11628\75781698.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat, 1)


In [11]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\peter\AppData\Local\Temp\ipykernel_11628\942049030.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'], 1).values


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hiZdden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               4200      
                                                                 
 dense_1 (Dense)             (None, 50)                6050      
                                                                 
 dense_2 (Dense)             (None, 18)                918       
                                                                 
 dense_3 (Dense)             (None, 1)                 19        
                                                                 
Total params: 11,187
Trainable params: 11,187
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [16]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6049 - accuracy: 0.6917
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5799 - accuracy: 0.7182
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5712 - accuracy: 0.7241
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5678 - accuracy: 0.7256
Epoch 5/200
771/804 [===========================>..] - ETA: 0s - loss: 0.5650 - accuracy: 0.7265
Epoch 5: saving model to optimized_checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5651 - accuracy: 0.7266
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5649 - accuracy: 0.7271
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5642 - accuracy: 0.7283
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5619 - accuracy: 0.7287
Epoch 9/200
804/804 [=======================

804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7342
Epoch 64/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7353
Epoch 65/200
541/804 [===================>..........] - ETA: 0s - loss: 0.5467 - accuracy: 0.7346
Epoch 65: saving model to optimized_checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7360
Epoch 66/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5464 - accuracy: 0.7362
Epoch 67/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7357
Epoch 68/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7355
Epoch 69/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7359
Epoch 70/200
518/804 [==================>...........] - ETA: 0s - loss: 0.5489 - accuracy: 0.7329
Epoch 70: saving model to optimized_checkpoints\weig

280/804 [=========>....................] - ETA: 0s - loss: 0.5374 - accuracy: 0.7425
Epoch 125: saving model to optimized_checkpoints\weights.125.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7385
Epoch 126/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5412 - accuracy: 0.7385
Epoch 127/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7378
Epoch 128/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5415 - accuracy: 0.7384
Epoch 129/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5414 - accuracy: 0.7382
Epoch 130/200
275/804 [=========>....................] - ETA: 0s - loss: 0.5334 - accuracy: 0.7468
Epoch 130: saving model to optimized_checkpoints\weights.130.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7390
Epoch 131/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accuracy

804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7392
Epoch 186/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7392
Epoch 187/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5388 - accuracy: 0.7394
Epoch 188/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7394
Epoch 189/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7396
Epoch 190/200
 36/804 [>.............................] - ETA: 1s - loss: 0.5358 - accuracy: 0.7352
Epoch 190: saving model to optimized_checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5385 - accuracy: 0.7397
Epoch 191/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5388 - accuracy: 0.7397
Epoch 192/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5386 - accuracy: 0.7399
Epoch 193/200
804/804 [================

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 1.5166 - accuracy: 0.6680 - 469ms/epoch - 2ms/step
Loss: 1.5166417360305786, Accuracy: 0.6679883599281311


In [18]:
# save as .h5 file
nn.save('AlphabetSoupCharity_Optimization.h5')